In [1]:
import torch
import numpy as np
import torch.nn as nn
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset,DataLoader
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype=torch.FloatTensor

In [2]:
sentences = ["jack like dog", "jack like cat", "jack like animal",
  "dog cat animal", "banana apple cat dog like", "dog fish milk like",
  "dog cat animal like", "jack like apple", "apple like", "jack like banana",
  "apple banana jack movie book music like", "cat dog hate", "cat dog like"]
sentences_list=" ".join(sentences).split()
vocab=list(set(sentences_list))
word2idx={w:i for i,w in enumerate(vocab)}
vocab_size=len(vocab)

In [3]:
# model parameters
C=2
batch_size=8
m=2

In [4]:
skip_grams=[]
for idx in range(C,len(sentences_list)-C):
    center=word2idx[sentences_list[idx]]
    context_idx=list(range(idx-C,idx))+list(range(idx+1,idx+C+1))
    context=[word2idx[sentences_list[i]] for i in context_idx]

    for w in context:
        skip_grams.append([center,w])

In [5]:
def make_data(skip_grams):
    x,y=[],[]
    for a,b in skip_grams:
        x.append(np.eye(vocab_size)[a])
        y.append(b)
    return x,y

In [6]:
x,y=make_data(skip_grams)
x,y=torch.Tensor(x),torch.LongTensor(y)
dataset=TensorDataset(x,y)
loader=DataLoader(dataset,batch_size,True)

C:\Users\mephistopheles\AppData\Local\Temp\ipykernel_15232\2563640646.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  x,y=torch.Tensor(x),torch.LongTensor(y)


In [7]:
class Word2Vec(nn.Module):
    def __init__(self):
        super(Word2Vec,self).__init__()
        self.W=nn.Parameter(torch.randn(vocab_size,m).type(dtype))
        self.V=nn.Parameter(torch.randn(m,vocab_size).type(dtype))
    def forward(self,x):
        hidden=torch.mm(x,self.W)
        output=torch.mm(hidden,self.V)
        return output

In [8]:
model=Word2Vec().to(device)
criterion=nn.CrossEntropyLoss()
optim=torch.optim.Adam(model.parameters(),lr=1e-3)

In [9]:
writer=SummaryWriter()
step=0
for epoch in range(10):
    loss_record=[]
    for batch_x,batch_y in tqdm(loader):
        batch_x,batch_y=batch_x.to(device),batch_y.to(device)
        pred=model(batch_x)
        loss=criterion(pred,batch_y)
        loss_record.append(loss)
        optim.zero_grad()
        loss.backward()
        optim.step()

        step+=1
    mean_train_loss=sum(loss_record)/len(loss_record)
    writer.add_scalar("trainloss",mean_train_loss,step)


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

In [10]:
import matplotlib.pyplot as plt
for i,label in enumerate(vocab):
    w,wt=model.parameters()
    print(w,wt)
    x,y=float(w[i][0]),float(w[i][1])
    print(x,y)
    plt.scatter(x,y)
    plt.annotate(label,xy=(x,y),xytext=(5,2))#,textcoords=)
plt.show()

Parameter containing:
tensor([[ 3.0464e-01,  1.6175e+00],
        [-2.0618e-01,  1.6635e+00],
        [-4.9500e-01,  2.8918e-01],
        [-2.0196e-01,  9.1923e-01],
        [-1.4689e+00,  2.0688e+00],
        [-3.0941e-01, -5.6723e-01],
        [ 1.9653e-01,  7.6729e-01],
        [-6.7791e-01,  1.0789e+00],
        [ 2.5823e-01,  7.6374e-01],
        [-6.5075e-01, -6.0112e-01],
        [-1.7608e+00,  7.8554e-04],
        [-1.3077e+00,  5.0368e-01],
        [-1.7800e+00,  7.3736e-02]], requires_grad=True) Parameter containing:
tensor([[-0.5806,  0.4541, -0.5058, -1.7984, -0.1472, -0.1695, -0.2166, -0.9295,
          1.7202, -1.9656, -0.4188, -0.1822, -0.8080],
        [ 0.1001,  0.5571, -1.0630,  0.2788,  1.5641, -0.2742, -0.3473, -1.2679,
          0.7953,  1.0516,  0.0766, -0.3992, -0.3814]], requires_grad=True)
0.30464044213294983 1.617525339126587


: 

: 